# Global Uncertainty Analysis: Polynomial Chaos Expansion (PCE) for Chemical Reaction System


This ipython notebook uses MUQ as a basis for adaptive Polynomial Chaos Expansions to perform global uncertainty analysis for chemical reaction systems.  This ipython notebook details a workflow using RMG, Cantera, and MUQ codes.  

In [ ]:
from rmgpy.tools.canteraModel import Cantera, getRMGSpeciesFromUserSpecies
from rmgpy.species import Species
from rmgpy.chemkin import loadChemkinFile
from rmgpy.tools.muq import ReactorPCEFactory

Load chemical kinetic mechanism from RMG chemkin file and dictionary.

In [ ]:
# Load chemkin file (truncated PDD model)
chemkinFile = 'data/minimal_model/chem_annotated.inp'
dictFile = 'data/minimal_model/species_dictionary.txt'

speciesList, reactionList = loadChemkinFile(chemkinFile, dictFile)

Create a Cantera model object that stores the reaction conditions.

In [ ]:
outputDir = 'uncertainty'

ethane=Species().fromSMILES('CC')
H = Species().fromSMILES('[H]')
C2H5 = Species().fromSMILES('C[CH2]')
C2H4 = Species().fromSMILES('C=C')

mapping = getRMGSpeciesFromUserSpecies([ethane, H, C2H5, C2H4], speciesList)

reactorTypeList = ['IdealGasConstPressureTemperatureReactor']
molFracList=[{mapping[ethane]: 1.0}]
Tlist = ([1200, 1300],'K')
Plist = ([1],'atm')
reactionTimeList = ([5e-4], 's')


job = Cantera(speciesList=speciesList, reactionList=reactionList, outputDirectory=outputDir)
# Load the cantera model based on the RMG reactions and species
job.loadModel()
# Generate the conditions based on the settings we declared earlier
job.generateConditions(reactorTypeList, reactionTimeList, molFracList, Tlist, Plist)


Input a set of kinetic $(k)$ and thermo $(G)$ parameters to be propagated and their uncertainties $(d\ln(k), dG)$ into the `ReactorPCEFactory` class.  These kinetic and thermo parameters should typically be pre-screened from local uncertainty analysis to narrow down to the most influential parameters.  
Each parameter's uncertainty is considered to be a uniform uncertainty interval where unit random variables $\ln(k)_{rv}$ and $G_{rv}$ are mapped to the user-assigned parameter uncertainties.

$\ln(k)_{rv} \sim U(-1, 1) \rightarrow ln(k) \sim U(-d\ln(k), d\ln(k))$

$G_{rv} \sim U(-1, 1) \rightarrow G \sim U(-dG, dG)$


Polynomial chaos expansions (PCE) are contructed for the desired outputs of interest (species mole fractions).




In [ ]:
# Create ReactorPCEFactory object
reactorPCEFactory = ReactorPCEFactory(cantera=job,
                            outputSpeciesList=[mapping[ethane], mapping[C2H4]],
                            kReactions=[7, 3],   # indices for the reactions in job.reactionList
                            kUncertainty=[2.0,2.0],   # dln(k)
                            gSpecies = [job.speciesList.index(mapping[H]), job.speciesList.index(mapping[C2H5]),
                                       job.speciesList.index(mapping[C2H4])],  # indices for the species in job.speciesList
                            gUncertainty=[1.0, 2.0, 2.0]   # dG in units of kcal/mol
                            )


Begin generating the PCEs adaptively based a runtime.

There are actually three methods for generating PCEs. See the `ReactorPCEFactory.generatePCE` function for more details.

- Option 1: Adaptive for a pre-specified amount of time
- Option 2: Adaptively construct PCE to error tolerance
- Option 3: Used a fixed order, and (optionally) adapt later.  

In [ ]:
reactorPCEFactory.generatePCE(runTime=60)  # runtime of 60 seconds.

Let's compare the outputs for a test point using the real model versus using the PCE approximation.
Evaluate the desired output mole fractions based on a set of inputs `ins = [[` $\ln(k)_{rv}$ `], [` $G_{rv}$ `]]` which contains the 
random unit uniform variables attributed to the uncertain kinetics and free energy parameters, respectively.

In [ ]:
trueTestPointOutput, pceTestPointOutput = reactorPCEFactory.compareOutput([0.5,0.2,0.1,-.5,.7])   

Obtain the results: the species mole fraction mean and variance computed from the PCE, as well as the global sensitivity indices.

In [ ]:
mean, variance, covariance, mainSens, totalSens = reactorPCEFactory.analyzeResults()